In [1]:
!pip install PyDesmos

  Preparing metadata (setup.py) ... done
  Created wheel for PyDesmos: filename=PyDesmos-0.1.3-py3-none-any.whl size=7764 sha256=d7626526255fb00853b26f4becadd73cd026f88d461d3394cead037d744b97f9
  Stored in directory: /root/.cache/pip/wheels/68/98/33/eb3bfa55e65ba4ae74b9a336cfa992a2962ef09a7488363fa3
Successfully built PyDesmos


In [2]:
import numpy as np
from PyDesmos import Graph

In [ ]:
# Generate the Random texture
import pandas as pd

def generate_cube_oriented_grains(n=200, spread_deg=5):
    """Generate Euler angles around (0,0,0) within ±spread_deg"""
    return np.random.uniform(low=-spread_deg, high=spread_deg, size=(n, 3))

def generate_random_grains(n=300, angle_range=(10, 30)):
    """Generate random Euler angles between 10 and 30 degrees"""
    low, high = angle_range
    return np.random.uniform(low=low, high=high, size=(n, 3))

def generate_500_grains():
    cube_grains = generate_cube_oriented_grains(200, spread_deg=5)
    random_grains = generate_random_grains(300, angle_range=(10, 69))
    all_grains = np.vstack([ cube_grains ,random_grains])

    df = pd.DataFrame(all_grains, columns=["phi1", "PHI", "phi2"])
    df.index.name = "Grain_ID"
    return df

# Generate and save
df_grains = generate_500_grains()
excel_file = "generated_500 random_euler_angles.xlsx"
df_grains.to_excel(excel_file)

print(f"Saved: {excel_file}")

In [3]:
import numpy as np

class Yieldlocus:
    def __init__(self, x, y, z, strain_external, bh_states):
        self.x = x
        self.y = y
        self.z = z
        self.bh_states = bh_states  # Pass the bh_states dictionary
        self.g_matrix = self.euler_to_g_matrix(x, y, z)
        self.strain_tensor = strain_external
        self.strain_internal = self.transform_strain()
        self.results = {}  # To store results from check_active_slip
        self.storing_TF = {} # To store Taylor Factors for filtered states
        self.highest_taylor_factor = None
        self.highest_taylor_system = None

        # Run the analysis upon initialization
        self.analyze_yield_locus()


    def euler_to_g_matrix(self, x, y, z):
        """ Compute the G matrix from Euler angles (in degrees) """
        phi1, phi, phi2 = np.radians([x, y, z])  # Convert to radians
        c1, s1 = np.cos(phi1), np.sin(phi1)
        c2, s2 = np.cos(phi2), np.sin(phi2)
        cP, sP = np.cos(phi), np.sin(phi)
        Gphi1 = np.array([
            [c1, s1, 0],
            [-s1, c1, 0],
            [0, 0, 1]
        ])
        Gphi = np.array([
            [1, 0, 0],
            [0, cP, sP],
            [0, -sP, cP]
        ])
        Gphi2 = np.array([
            [c2, s2, 0],
            [-s2, c2, 0],
            [0, 0, 1]
        ])
        G = Gphi2 @ Gphi @ Gphi1
        return G

    def transform_strain(self):
        return np.matmul(np.matmul(self.g_matrix, self.strain_tensor), self.g_matrix.T)

    def compute_taylor_factor(self, A, B, C, F, G, H, strain_internal, exx):
        M = 2.44 * (-B * strain_internal[0, 0] / exx + A * strain_internal[1, 1] / exx +
                         2 * F * strain_internal[1, 2] / exx + 2 * G * strain_internal[0, 2] / exx +
                         2 * H * strain_internal[0, 1] / exx)
        return M

    def check_active_slip(self, bh_state_array):
        """
        Checks active slip systems for a given Bishop-Hill state (NumPy array).

        Args:
            bh_state_array: A NumPy array of shape (6,) representing the
                            Bishop-Hill state [A, B, C, F, G, H].

        Returns:
            A tuple containing:
                - A dictionary of active slip systems and their values (+1 or -1).
                - The count of active slip systems.
        """
        A, B, C, F, G, H = bh_state_array

        values = {
            "a1": A - G + H, "a2": B + F - H, "a3": C - F + G,
            "b1": A + G + H, "b2": B - F - H, "b3": C + F - G,
            "c1": A + G - H, "c2": B + F + H, "c3": C - F - G,
            "d1": A - G - H, "d2": B - F + H, "d3": C + F + G
        }

        # Check which values are +1 or -1 using NumPy's close function
        active_slips = {key: val for key, val in values.items() if np.isclose(abs(val), 1)}
        count_active = len(active_slips)
        return active_slips, count_active

    def analyze_yield_locus(self):
        """
        Performs the analysis to find active slip systems and Taylor Factors.
        """
        # Iterate through all BH states and check the conditions
        for state, bh_array in self.bh_states.items():
            active_slips, count_active = self.check_active_slip(bh_array)
            if active_slips:
                self.results[state] = (active_slips, count_active)

        # Filter for states with 5 or more active slip systems and compute Taylor Factor
        exx = 1.0  # Assuming exx is constant for this calculation

        max_M = float("-inf") # Initialize maximum Taylor Factor

        for bh_state, (active_slips_dict, count) in self.results.items():
            if count >= 5:
                # Extract A, B, C, F, G, H directly from the bh_states dictionary
                A, B, C, F, G, H = self.bh_states[bh_state]

                # Compute Taylor Factor
                M = self.compute_taylor_factor(A, B, C, F, G, H, self.strain_internal, exx)

                # Store the result
                self.storing_TF[bh_state] = ((active_slips_dict, count), M)

                # Update the highest Taylor Factor if the current one is greater
                if M > max_M:
                  max_M = M
                  self.highest_taylor_factor = M
                  self.highest_taylor_system = bh_state # Store the state with the highest M

        # Optional: Sort storing_TF by Taylor Factor (M) in descending order
        self.sorted_TF = sorted(self.storing_TF.items(), key=lambda x: x[1][1], reverse=True)


# Example Usage (assuming bh_states is defined as a dictionary of NumPy arrays):
# bh_states = { ... np.array([...]), ... }
# strain_tensor = np.array([[...], [...], [...]]) # Define your strain tensor
bh_states = {
    "BH_1":  np.array([1, -1, 0, 0, 0, 0]), "BH_2":  np.array([0, 1, -1, 0, 0, 0]), "BH_3":  np.array([-1, 0, 1, 0, 0, 0]),
    "BH_4":  np.array([0, 0, 0, 1, 0, 0]), "BH_5":  np.array([0, 0, 0, 0, 1, 0]), "BH_6":  np.array([0, 0, 0, 0, 0, 1]),
    "BH_7":  np.array([0.5, -1, 0.5, 0, 0.5, 0]), "BH_8":  np.array([0.5, -1, 0.5, 0, -0.5, 0]),
    "BH_9":  np.array([-1, 0.5, 0.5, 0.5, 0, 0]), "BH_10": np.array([-1, 0.5, 0.5, -0.5, 0, 0]),
    "BH_11": np.array([0.5, 0.5, -1, 0, 0, 0.5]), "BH_12": np.array([0.5, 0.5, -1, 0, 0, -0.5]),
    "BH_13": np.array([0.5, 0, -0.5, 0.5, 0, 0.5]), "BH_14": np.array([0.5, 0, -0.5, -0.5, 0, 0.5]),
    "BH_15": np.array([0.5, 0, -0.5, 0.5, 0, -0.5]), "BH_16": np.array([0.5, 0, -0.5, -0.5, 0, -0.5]),
    "BH_17": np.array([0, -0.5, 0.5, 0, 0.5, 0.5]), "BH_18": np.array([0, -0.5, 0.5, 0, -0.5, 0.5]),
    "BH_19": np.array([0, -0.5, 0.5, 0, 0.5, -0.5]), "BH_20": np.array([0, -0.5, 0.5, 0, -0.5, -0.5]),
    "BH_21": np.array([-0.5, 0.5, 0, 0.5, 0, 0]), "BH_22": np.array([-0.5, 0.5, 0, -0.5, 0, 0]),
    "BH_23": np.array([-0.5, 0.5, 0, 0, -0.5, 0]), "BH_24": np.array([-0.5, 0.5, 0, 0, 0.5, 0]),
    "BH_25": np.array([0, 0, 0, 0.5, 0.5, 0]), "BH_26": np.array([0, 0, 0, 0.5, -0.5, 0]),
    "BH_27": np.array([0, 0, 0, -0.5, 0.5, 0]), "BH_28": np.array([0, 0, 0, -0.5, -0.5, 0]),

    "BH_29": np.array([-1, 1, 0, 0, 0, 0]), "BH_30": np.array([0, -1, 1, 0, 0, 0]), "BH_31": np.array([1, 0, -1, 0, 0, 0]),
    "BH_32": np.array([0, 0, 0, -1, 0, 0]), "BH_33": np.array([0, 0, 0, 0, -1, 0]), "BH_34": np.array([0, 0, 0, 0, 0, -1]),
    "BH_35": np.array([-0.5, 1, -0.5, 0, -0.5, 0]), "BH_36": np.array([-0.5, 1, -0.5, 0, 0.5, 0]),
    "BH_37": np.array([1, -0.5, -0.5, -0.5, 0, 0]), "BH_38": np.array([1, -0.5, -0.5, 0.5, 0, 0]),
    "BH_39": np.array([-0.5, -0.5, 1, 0, 0, -0.5]), "BH_40": np.array([-0.5, -0.5, 1, 0, 0, 0.5]),
    "BH_41": np.array([-0.5, 0, 0.5, -0.5, 0, -0.5]), "BH_42": np.array([-0.5, 0, 0.5, 0.5, 0, -0.5]),
    "BH_43": np.array([-0.5, 0, 0.5, -0.5, 0, 0.5]), "BH_44": np.array([-0.5, 0, 0.5, 0.5, 0, 0.5]),
    "BH_45": np.array([0, 0.5, -0.5, 0, -0.5, -0.5]), "BH_46": np.array([0, 0.5, -0.5, 0, 0.5, -0.5]),
    "BH_47": np.array([0, 0.5, -0.5, 0, -0.5, 0.5]), "BH_48": np.array([0, 0.5, -0.5, 0, 0.5, 0.5]),
    "BH_49": np.array([0.5, -0.5, 0, -0.5, -0.5, 0]), "BH_50": np.array([0.5, -0.5, 0, 0.5, -0.5, 0]),
    "BH_51": np.array([0.5, -0.5, 0, -0.5, 0.5, 0]), "BH_52": np.array([0.5, -0.5, 0, 0.5, 0.5, 0]),
    "BH_53": np.array([0, 0, 0, -0.5, -0.5, 0.5]), "BH_54": np.array([0, 0, 0, -0.5, 0.5, -0.5]),
    "BH_55": np.array([0, 0, 0, 0.5, -0.5, -0.5]), "BH_56": np.array([0, 0, 0, -0.5, -0.5, -0.5])
}


strain_tensor = np.array([
    [1,0.0, 0.0],
    [0.0, 0.0, -0.0],
    [0.00, -0.0, -1.0]
])


# Create an instance of the class
yield_locus_analysis = Yieldlocus(59, 37, 63, strain_tensor, bh_states)

# Access the results
print("Results (Active Slips and Count):", yield_locus_analysis.results)
print("\nStoring Taylor Factors (Filtered States):", yield_locus_analysis.storing_TF)
print("\nSorted Taylor Factors:", yield_locus_analysis.sorted_TF)
print("\nHighest Taylor Factor:", yield_locus_analysis.highest_taylor_factor)
print("Slip System with Highest Taylor Factor:", yield_locus_analysis.highest_taylor_system)

Results (Active Slips and Count): {'BH_1': ({'a1': np.int64(1), 'a2': np.int64(-1), 'b1': np.int64(1), 'b2': np.int64(-1), 'c1': np.int64(1), 'c2': np.int64(-1), 'd1': np.int64(1), 'd2': np.int64(-1)}, 8), 'BH_2': ({'a2': np.int64(1), 'a3': np.int64(-1), 'b2': np.int64(1), 'b3': np.int64(-1), 'c2': np.int64(1), 'c3': np.int64(-1), 'd2': np.int64(1), 'd3': np.int64(-1)}, 8), 'BH_3': ({'a1': np.int64(-1), 'a3': np.int64(1), 'b1': np.int64(-1), 'b3': np.int64(1), 'c1': np.int64(-1), 'c3': np.int64(1), 'd1': np.int64(-1), 'd3': np.int64(1)}, 8), 'BH_4': ({'a2': np.int64(1), 'a3': np.int64(-1), 'b2': np.int64(-1), 'b3': np.int64(1), 'c2': np.int64(1), 'c3': np.int64(-1), 'd2': np.int64(-1), 'd3': np.int64(1)}, 8), 'BH_5': ({'a1': np.int64(-1), 'a3': np.int64(1), 'b1': np.int64(1), 'b3': np.int64(-1), 'c1': np.int64(1), 'c3': np.int64(-1), 'd1': np.int64(-1), 'd3': np.int64(1)}, 8), 'BH_6': ({'a1': np.int64(1), 'a2': np.int64(-1), 'b1': np.int64(1), 'b2': np.int64(-1), 'c1': np.int64(-1), 'c

In [37]:
## For single crystal

import numpy as np
from PyDesmos import Graph

# Assume your Yeildlocus class is defined above
# Make sure you have the bh_states dictionary defined (as in previous examples)

# Define your Bishop-Hill states as a dictionary of NumPy arrays
bh_states = {
    "BH_1":  np.array([1, -1, 0, 0, 0, 0]), "BH_2":  np.array([0, 1, -1, 0, 0, 0]), "BH_3":  np.array([-1, 0, 1, 0, 0, 0]),
    "BH_4":  np.array([0, 0, 0, 1, 0, 0]), "BH_5":  np.array([0, 0, 0, 0, 1, 0]), "BH_6":  np.array([0, 0, 0, 0, 0, 1]),
    "BH_7":  np.array([0.5, -1, 0.5, 0, 0.5, 0]), "BH_8":  np.array([0.5, -1, 0.5, 0, -0.5, 0]),
    "BH_9":  np.array([-1, 0.5, 0.5, 0.5, 0, 0]), "BH_10": np.array([-1, 0.5, 0.5, -0.5, 0, 0]),
    "BH_11": np.array([0.5, 0.5, -1, 0, 0, 0.5]), "BH_12": np.array([0.5, 0.5, -1, 0, 0, -0.5]),
    "BH_13": np.array([0.5, 0, -0.5, 0.5, 0, 0.5]), "BH_14": np.array([0.5, 0, -0.5, -0.5, 0, 0.5]),
    "BH_15": np.array([0.5, 0, -0.5, 0.5, 0, -0.5]), "BH_16": np.array([0.5, 0, -0.5, -0.5, 0, -0.5]),
    "BH_17": np.array([0, -0.5, 0.5, 0, 0.5, 0.5]), "BH_18": np.array([0, -0.5, 0.5, 0, -0.5, 0.5]),
    "BH_19": np.array([0, -0.5, 0.5, 0, 0.5, -0.5]), "BH_20": np.array([0, -0.5, 0.5, 0, -0.5, -0.5]),
    "BH_21": np.array([-0.5, 0.5, 0, 0.5, 0, 0]), "BH_22": np.array([-0.5, 0.5, 0, -0.5, 0, 0]),
    "BH_23": np.array([-0.5, 0.5, 0, 0, -0.5, 0]), "BH_24": np.array([-0.5, 0.5, 0, 0, 0.5, 0]),
    "BH_25": np.array([0, 0, 0, 0.5, 0.5, 0]), "BH_26": np.array([0, 0, 0, 0.5, -0.5, 0]),
    "BH_27": np.array([0, 0, 0, -0.5, 0.5, 0]), "BH_28": np.array([0, 0, 0, -0.5, -0.5, 0]),

    "BH_29": np.array([-1, 1, 0, 0, 0, 0]), "BH_30": np.array([0, -1, 1, 0, 0, 0]), "BH_31": np.array([1, 0, -1, 0, 0, 0]),
    "BH_32": np.array([0, 0, 0, -1, 0, 0]), "BH_33": np.array([0, 0, 0, 0, -1, 0]), "BH_34": np.array([0, 0, 0, 0, 0, -1]),
    "BH_35": np.array([-0.5, 1, -0.5, 0, -0.5, 0]), "BH_36": np.array([-0.5, 1, -0.5, 0, 0.5, 0]),
    "BH_37": np.array([1, -0.5, -0.5, -0.5, 0, 0]), "BH_38": np.array([1, -0.5, -0.5, 0.5, 0, 0]),
    "BH_39": np.array([-0.5, -0.5, 1, 0, 0, -0.5]), "BH_40": np.array([-0.5, -0.5, 1, 0, 0, 0.5]),
    "BH_41": np.array([-0.5, 0, 0.5, -0.5, 0, -0.5]), "BH_42": np.array([-0.5, 0, 0.5, 0.5, 0, -0.5]),
    "BH_43": np.array([-0.5, 0, 0.5, -0.5, 0, 0.5]), "BH_44": np.array([-0.5, 0, 0.5, 0.5, 0, 0.5]),
    "BH_45": np.array([0, 0.5, -0.5, 0, -0.5, -0.5]), "BH_46": np.array([0, 0.5, -0.5, 0, 0.5, -0.5]),
    "BH_47": np.array([0, 0.5, -0.5, 0, -0.5, 0.5]), "BH_48": np.array([0, 0.5, -0.5, 0, 0.5, 0.5]),
    "BH_49": np.array([0.5, -0.5, 0, -0.5, -0.5, 0]), "BH_50": np.array([0.5, -0.5, 0, 0.5, -0.5, 0]),
    "BH_51": np.array([0.5, -0.5, 0, -0.5, 0.5, 0]), "BH_52": np.array([0.5, -0.5, 0, 0.5, 0.5, 0]),
    "BH_53": np.array([0, 0, 0, -0.5, -0.5, 0.5]), "BH_54": np.array([0, 0, 0, -0.5, 0.5, -0.5]),
    "BH_55": np.array([0, 0, 0, 0.5, -0.5, -0.5]), "BH_56": np.array([0, 0, 0, -0.5, -0.5, -0.5])
}

# Euler angles for the Yeildlocus class (replace with your desired values)
euler_x, euler_y, euler_z = 0, 0, 0

# --- Analysis for varying 'start' ---
# Define the range for the 'start' variable
start_values = np.linspace(0, 1, 100) # Example: 100 points between 0 and 1

# Dictionary to store highest M and slip system for each 'start'
M_by_start = {}

for current_start in start_values:
    # Define the strain tensor based on the current 'start' value
    strain_tensor = np.array([
        [1, 0.0, 0.0],
        [0.0, -current_start, 0.0],
        [0.0, 0.0, -1 + current_start]
    ])

    # Create an instance of the Yeildlocus class for the current strain tensor
    yield_locus_analysis = Yieldlocus(euler_x, euler_y, euler_z, strain_tensor, bh_states)

    # Store the highest Taylor Factor and the corresponding system
    M_by_start[current_start] = (yield_locus_analysis.highest_taylor_system,
                                yield_locus_analysis.highest_taylor_factor)

# Print results for varying 'start' (optional)
print("\nStart Values with Corresponding Highest M:")
for s, (sys, m) in M_by_start.items():
    print(f"start = {s:.4f} -> Slip System {sys}, M = {m:.4f}")


# --- Analysis for varying 'phro' ---
# Define the range for 'phro'
phro_values = np.linspace(100, 1.01, 100) # 100 points from 100 down to 1.01

# Dictionary to store highest M and slip system for each 'phro'
M_by_phro = {}

for current_phro in phro_values:
    # Define the strain tensor based on the current 'phro' value
    # Check to prevent division by zero if phro is exactly 0
    if current_phro != 0:
        strain_tensor = np.array([
            [-1 / current_phro, 0.0, 0.0],
            [0.0, 1, 0.0],
            [0.0, 0.0, - (current_phro - 1) / current_phro]
        ])
    else:
        # Skip if phro is 0 to avoid division by zero.
        continue

    # Create an instance of the Yeildlocus class for the current strain tensor
    yield_locus_analysis = Yieldlocus(euler_x, euler_y, euler_z, strain_tensor, bh_states)

    # Store the highest Taylor Factor and the corresponding system
    M_by_phro[current_phro] = (yield_locus_analysis.highest_taylor_system,
                                yield_locus_analysis.highest_taylor_factor)

# Print results for varying 'phro' (optional)
print("\nPhro Values with Corresponding Highest M:")
for p, (sys, m) in M_by_phro.items():
    print(f"phro = {p:.4f} -> Slip System {sys}, M = {m:.4f}")

# ---- Analysis for z
M_by_z={}

z_phro_values = np.linspace(-99, -0.01, 100)

for current_z in z_phro_values:
  if current_z != 0:
    strain_tensor = np.array([
            [1 / (current_z-1), 0.0, 0.0],
            [0.0, - current_z / (current_z -1) , 0.0],
            [0.0, 0.0, 1]
        ])
  else:
      continue
  yield_locus_analysis = Yieldlocus(euler_x, euler_y, euler_z, strain_tensor, bh_states)

  M_by_z[current_z] = (yield_locus_analysis.highest_taylor_system,
                                yield_locus_analysis.highest_taylor_factor)

print("\nZ Values with Corresponding Highest M:")
for z, (sys, m) in M_by_z.items():
    print(f"z = {z:.4f} -> Slip System {sys}, M = {m:.4f}")



# --- PyDemos Plotting ---

with Graph('Yield Locus for Cube Single') as G:
    # Plot lines for varying 'start'
    for s, (sys, highest_taylor_factor) in M_by_start.items():
        # Equations: x - phoy = RHS and -x + phoy = RHS
        # Rearranged to solve for y (assuming x on x-axis, y on y-axis):
        # y = (x - RHS) / pho
        # y = (x + RHS) / pho

        # Assuming 'pho' for this set of lines is related to 'start'
        pho_coeff = s

        # Calculate RHS for this strain state
        RHS = highest_taylor_factor / np.sqrt(6)

        # Avoid division by zero if pho_coeff is very close to zero
        if not np.isclose(pho_coeff, 0):
            # Add expressions to the PyDemos graph using G.append()
            # Expression 1: y = (x - RHS) / pho_coeff
            G.append(f'y = (x - {RHS}) / {pho_coeff}')
            # Expression 2: y = (x + RHS) / pho_coeff
            G.append(f'y = (x + {RHS}) / {pho_coeff}')
        else:
            # If pho is zero, the equations become x = RHS and x = -RHS (vertical lines)
            G.append(f'x = {RHS}')
            G.append(f'x = {-RHS}')


    # Plot lines for varying 'phro'
    for phro_val, (sys, highest_taylor_factor) in M_by_phro.items():
        # Equations: -x/pho + y = RHS and x/pho - y = RHS
        # Rearranged to solve for y (assuming x on x-axis, y on y-axis):
        # y = RHS + x / pho
        # y = -RHS + x / pho

        # Assuming 'pho' for this set of lines is related to 'phro'
        pho_coeff = phro_val

        # Calculate RHS for this strain state
        RHS = highest_taylor_factor / np.sqrt(6)

        # Ensure pho_coeff is not zero to avoid division by zero in the equation
        if pho_coeff != 0:
             # Add expressions to the PyDemos graph using G.append()
            # Expression 1: y = RHS + x / pho_coeff
            G.append(f'y = {RHS} + x / {pho_coeff}')
            # Expression 2: y = -RHS + x / pho_coeff
            G.append(f'y = {-RHS} + x / {pho_coeff}')
        # Note: If pho_coeff is 0 here, the equations become y = RHS and y = -RHS (horizontal lines).
        # However, the strain tensor definition for phro includes 1/phro, so phro=0 is likely
        # a physically invalid or singular case you are already skipping.
        # If you need to handle the phro=0 case for plotting, you would add
        # G.append(f'y = {RHS}') and G.append(f'y = {-RHS}') here.

    # Plot for varing Z
    for z, (sys, highest_taylor_factor) in M_by_z.items():
        # Equations: x - phoy = RHS and -x + phoy = RHS
        # Rearranged to solve for y (assuming x on x-axis, y on y-axis):
        RHS= highest_taylor_factor / np.sqrt(6)
        if not np.isclose(z, 0):
            G.append(f'y = (x - {RHS} * ({z} - 1)) / {z}')

            G.append(f'y = (x + {RHS} * ({z} - 1)) / {z}')



# The PyDemos graph will be generated when the 'with' block exits.
# You can then open the generated HTML file to view the graph in Desmos.


Start Values with Corresponding Highest M:
start = 0.0000 -> Slip System BH_1, M = 2.4400
start = 0.0101 -> Slip System BH_30, M = 2.4400
start = 0.0202 -> Slip System BH_30, M = 2.4400
start = 0.0303 -> Slip System BH_30, M = 2.4400
start = 0.0404 -> Slip System BH_30, M = 2.4400
start = 0.0505 -> Slip System BH_30, M = 2.4400
start = 0.0606 -> Slip System BH_30, M = 2.4400
start = 0.0707 -> Slip System BH_30, M = 2.4400
start = 0.0808 -> Slip System BH_30, M = 2.4400
start = 0.0909 -> Slip System BH_30, M = 2.4400
start = 0.1010 -> Slip System BH_30, M = 2.4400
start = 0.1111 -> Slip System BH_30, M = 2.4400
start = 0.1212 -> Slip System BH_30, M = 2.4400
start = 0.1313 -> Slip System BH_30, M = 2.4400
start = 0.1414 -> Slip System BH_30, M = 2.4400
start = 0.1515 -> Slip System BH_30, M = 2.4400
start = 0.1616 -> Slip System BH_30, M = 2.4400
start = 0.1717 -> Slip System BH_30, M = 2.4400
start = 0.1818 -> Slip System BH_30, M = 2.4400
start = 0.1919 -> Slip System BH_30, M = 2.44

In [4]:
import pandas as pd
import numpy as np

In [14]:
# # Read Random Texture- 500
# df=pd.read_excel('/content/generated_500 random_euler_angles - Copy.xlsx')

In [15]:
# df.head()

,Phi1,Phi,Phi2
0,39.989624,19.437643,62.018649
1,44.086785,55.591918,40.628457
2,40.419153,37.954673,36.305885
3,42.771567,43.975645,14.794039
4,65.527524,22.129404,26.278627


In [41]:
import pandas as pd
import numpy as np # You already have this imported in your notebook

# Assuming your Excel file is in the correct path
df = pd.read_excel('/content/generated_500_euler_angles.xlsx')

# The code to create the list of tuples
phi_tuples_list = []

for row in df.itertuples(index=False):
  # print(row.Ph1)
    phi1_value = row.Phi1
    phi_value = row.Phi
    phi2_value = row.Phi2

    phi_tuples_list.append((phi1_value, phi_value, phi2_value))

# Now you have the list of tuples
print("List of (phi1, phi, phi2) tuples:", phi_tuples_list)

List of (phi1, phi, phi2) tuples: [(-3.558643382085584, -0.4852544786509219, 2.772433142273694), (-1.871703946524563, 3.934418600235606, 4.982607161909097), (-3.173422593123614, 1.132206861273831, 1.121510242986639), (1.86969595039657, 0.3434680150357909, -2.892204835340675), (-1.602607589595815, -0.3963160133955217, -3.454289905258177), (-4.863283499493317, 3.409389039839168, 4.019668012038661), (3.436182683860181, -0.9050366926205591, -1.776551063684085), (-3.226181303836585, -0.8429536567607503, 2.63077350423152), (2.306784459801535, 3.138634325446354, 2.25005636767091), (2.297085504552813, -2.164865562364253, 2.298869413272891), (1.915032323931507, -4.249940711810408, -3.114624342699578), (-1.603155505707549, 3.918585863327253, -4.722023990425025), (-3.451884949572976, -0.3919871160919053, 2.080042681259111), (-0.457575564554876, 4.387628859175638, 0.4790255463837267), (3.629146443341201, -4.815492697523195, 0.8182894648251438), (1.89044700374139, -2.542756230787585, -4.95541805054

In [42]:
phi_tuples_list

[(-3.558643382085584, -0.4852544786509219, 2.772433142273694),
 (-1.871703946524563, 3.934418600235606, 4.982607161909097),
 (-3.173422593123614, 1.132206861273831, 1.121510242986639),
 (1.86969595039657, 0.3434680150357909, -2.892204835340675),
 (-1.602607589595815, -0.3963160133955217, -3.454289905258177),
 (-4.863283499493317, 3.409389039839168, 4.019668012038661),
 (3.436182683860181, -0.9050366926205591, -1.776551063684085),
 (-3.226181303836585, -0.8429536567607503, 2.63077350423152),
 (2.306784459801535, 3.138634325446354, 2.25005636767091),
 (2.297085504552813, -2.164865562364253, 2.298869413272891),
 (1.915032323931507, -4.249940711810408, -3.114624342699578),
 (-1.603155505707549, 3.918585863327253, -4.722023990425025),
 (-3.451884949572976, -0.3919871160919053, 2.080042681259111),
 (-0.457575564554876, 4.387628859175638, 0.4790255463837267),
 (3.629146443341201, -4.815492697523195, 0.8182894648251438),
 (1.89044700374139, -2.542756230787585, -4.955418050546113),
 (-1.4211092

In [43]:
len(phi_tuples_list)

500

In [44]:
import numpy as np
from PyDesmos import Graph

# Assume your Yeildlocus class is defined above
# Make sure you have the bh_states dictionary defined (as in previous examples)

# Define your Bishop-Hill states as a dictionary of NumPy arrays
bh_states = {
    "BH_1":  np.array([1, -1, 0, 0, 0, 0]), "BH_2":  np.array([0, 1, -1, 0, 0, 0]), "BH_3":  np.array([-1, 0, 1, 0, 0, 0]),
    "BH_4":  np.array([0, 0, 0, 1, 0, 0]), "BH_5":  np.array([0, 0, 0, 0, 1, 0]), "BH_6":  np.array([0, 0, 0, 0, 0, 1]),
    "BH_7":  np.array([0.5, -1, 0.5, 0, 0.5, 0]), "BH_8":  np.array([0.5, -1, 0.5, 0, -0.5, 0]),
    "BH_9":  np.array([-1, 0.5, 0.5, 0.5, 0, 0]), "BH_10": np.array([-1, 0.5, 0.5, -0.5, 0, 0]),
    "BH_11": np.array([0.5, 0.5, -1, 0, 0, 0.5]), "BH_12": np.array([0.5, 0.5, -1, 0, 0, -0.5]),
    "BH_13": np.array([0.5, 0, -0.5, 0.5, 0, 0.5]), "BH_14": np.array([0.5, 0, -0.5, -0.5, 0, 0.5]),
    "BH_15": np.array([0.5, 0, -0.5, 0.5, 0, -0.5]), "BH_16": np.array([0.5, 0, -0.5, -0.5, 0, -0.5]),
    "BH_17": np.array([0, -0.5, 0.5, 0, 0.5, 0.5]), "BH_18": np.array([0, -0.5, 0.5, 0, -0.5, 0.5]),
    "BH_19": np.array([0, -0.5, 0.5, 0, 0.5, -0.5]), "BH_20": np.array([0, -0.5, 0.5, 0, -0.5, -0.5]),
    "BH_21": np.array([-0.5, 0.5, 0, 0.5, 0, 0]), "BH_22": np.array([-0.5, 0.5, 0, -0.5, 0, 0]),
    "BH_23": np.array([-0.5, 0.5, 0, 0, -0.5, 0]), "BH_24": np.array([-0.5, 0.5, 0, 0, 0.5, 0]),
    "BH_25": np.array([0, 0, 0, 0.5, 0.5, 0]), "BH_26": np.array([0, 0, 0, 0.5, -0.5, 0]),
    "BH_27": np.array([0, 0, 0, -0.5, 0.5, 0]), "BH_28": np.array([0, 0, 0, -0.5, -0.5, 0]),

    "BH_29": np.array([-1, 1, 0, 0, 0, 0]), "BH_30": np.array([0, -1, 1, 0, 0, 0]), "BH_31": np.array([1, 0, -1, 0, 0, 0]),
    "BH_32": np.array([0, 0, 0, -1, 0, 0]), "BH_33": np.array([0, 0, 0, 0, -1, 0]), "BH_34": np.array([0, 0, 0, 0, 0, -1]),
    "BH_35": np.array([-0.5, 1, -0.5, 0, -0.5, 0]), "BH_36": np.array([-0.5, 1, -0.5, 0, 0.5, 0]),
    "BH_37": np.array([1, -0.5, -0.5, -0.5, 0, 0]), "BH_38": np.array([1, -0.5, -0.5, 0.5, 0, 0]),
    "BH_39": np.array([-0.5, -0.5, 1, 0, 0, -0.5]), "BH_40": np.array([-0.5, -0.5, 1, 0, 0, 0.5]),
    "BH_41": np.array([-0.5, 0, 0.5, -0.5, 0, -0.5]), "BH_42": np.array([-0.5, 0, 0.5, 0.5, 0, -0.5]),
    "BH_43": np.array([-0.5, 0, 0.5, -0.5, 0, 0.5]), "BH_44": np.array([-0.5, 0, 0.5, 0.5, 0, 0.5]),
    "BH_45": np.array([0, 0.5, -0.5, 0, -0.5, -0.5]), "BH_46": np.array([0, 0.5, -0.5, 0, 0.5, -0.5]),
    "BH_47": np.array([0, 0.5, -0.5, 0, -0.5, 0.5]), "BH_48": np.array([0, 0.5, -0.5, 0, 0.5, 0.5]),
    "BH_49": np.array([0.5, -0.5, 0, -0.5, -0.5, 0]), "BH_50": np.array([0.5, -0.5, 0, 0.5, -0.5, 0]),
    "BH_51": np.array([0.5, -0.5, 0, -0.5, 0.5, 0]), "BH_52": np.array([0.5, -0.5, 0, 0.5, 0.5, 0]),
    "BH_53": np.array([0, 0, 0, -0.5, -0.5, 0.5]), "BH_54": np.array([0, 0, 0, -0.5, 0.5, -0.5]),
    "BH_55": np.array([0, 0, 0, 0.5, -0.5, -0.5]), "BH_56": np.array([0, 0, 0, -0.5, -0.5, -0.5])
}

# Euler angles for the Yeildlocus class (replace with your desired values)
# euler_x, euler_y, euler_z = 0, 45, 0

# --- Analysis for varying 'start' ---
# Define the range for the 'start' variable
start_values = np.linspace(0, 1, 100) # Example: 100 points between 0 and 1

# Dictionary to store highest M and slip system for each 'start'
M_by_start = {}

for current_start in start_values:
    # Define the strain tensor based on the current 'start' value
    strain_tensor = np.array([
        [1, 0.0, 0.0],
        [0.0, -current_start, 0.0],
        [0.0, 0.0, -1 + current_start]
    ])

    highest_taylor_factors_list = []

    # Create an instance of the Yeildlocus class for the current strain tensor
    for current_phi1, current_phi, current_phi2 in phi_tuples_list:
      euler_x, euler_y, euler_z = current_phi1, current_phi, current_phi2
      yield_locus_analysis = Yieldlocus(euler_x, euler_y, euler_z, strain_tensor, bh_states)
      highest_taylor_factors_list.append(yield_locus_analysis.highest_taylor_factor)

    avg_TF=np.mean(highest_taylor_factors_list)
    print("Avg value",avg_TF)

    # Store the highest Taylor Factor and the corresponding system
    M_by_start[current_start] = (yield_locus_analysis.highest_taylor_system,
                                avg_TF)

# Print results for varying 'start' (optional)
print("\nStart Values with Corresponding Highest M:")
for s, (sys, m) in M_by_start.items():
    print(f"start = {s:.4f} -> Slip System {sys}, M = {m:.4f}")


# --- Analysis for varying 'phro' ---
# Define the range for 'phro'
phro_values = np.linspace(100, 1.01, 100) # 100 points from 100 down to 1.01

# Dictionary to store highest M and slip system for each 'phro' Y=1
M_by_phro = {}

for current_phro in phro_values:
    # Define the strain tensor based on the current 'phro' value
    # Check to prevent division by zero if phro is exactly 0
    if current_phro != 0:
        strain_tensor = np.array([
            [-1 / current_phro, 0.0, 0.0],
            [0.0, 1, 0.0],
            [0.0, 0.0, - (current_phro - 1) / current_phro]
        ])
    else:
        # Skip if phro is 0 to avoid division by zero.
        continue

    # Create an instance of the Yeildlocus class for the current strain tensor
    highest_taylor_factors_list = []

    # Create an instance of the Yeildlocus class for the current strain tensor
    for current_phi1, current_phi, current_phi2 in phi_tuples_list:
      euler_x, euler_y, euler_z = current_phi1, current_phi, current_phi2
      yield_locus_analysis = Yieldlocus(euler_x, euler_y, euler_z, strain_tensor, bh_states)
      highest_taylor_factors_list.append(yield_locus_analysis.highest_taylor_factor)

    avg_TF=np.mean(highest_taylor_factors_list)
    print("Avg value",avg_TF)

    # Store the highest Taylor Factor and the corresponding system
    M_by_phro[current_phro] = (yield_locus_analysis.highest_taylor_system,
                                avg_TF)

# Print results for varying 'phro' (optional)
print("\nPhro Values with Corresponding Highest M:")
for p, (sys, m) in M_by_phro.items():
    print(f"phro = {p:.4f} -> Slip System {sys}, M = {m:.4f}")

# ---- Analysis for z  =1
M_by_z={}

z_phro_values = np.linspace(-99, -0.01, 100)

for current_z in z_phro_values:
  if current_z != 0:
    strain_tensor = np.array([
            [1 / (current_z-1), 0.0, 0.0],
            [0.0, - current_z / (current_z -1) , 0.0],
            [0.0, 0.0, 1]
        ])


    highest_taylor_factors_list = []

    # Create an instance of the Yeildlocus class for the current strain tensor
    for current_phi1, current_phi, current_phi2 in phi_tuples_list:
        euler_x, euler_y, euler_z = current_phi1, current_phi, current_phi2
        yield_locus_analysis = Yieldlocus(euler_x, euler_y, euler_z, strain_tensor, bh_states)
        highest_taylor_factors_list.append(yield_locus_analysis.highest_taylor_factor)

    avg_TF=np.mean(highest_taylor_factors_list)
    print("Avg value",avg_TF)


    # Store the highest Taylor Factor and the corresponding system
    M_by_z[current_z] = (yield_locus_analysis.highest_taylor_system,
                                avg_TF)

print("\nZ Values with Corresponding Highest M:")
for z, (sys, m) in M_by_z.items():
    print(f"z = {z:.4f} -> Slip System {sys}, M = {m:.4f}")



# --- PyDemos Plotting ---

with Graph('Yield Locus for Cube Polycrystal') as G:
    # Plot lines for varying 'start'
    for s, (sys, highest_taylor_factor) in M_by_start.items():
        # Equations: x - phoy = RHS and -x + phoy = RHS
        # Rearranged to solve for y (assuming x on x-axis, y on y-axis):
        # y = (x - RHS) / pho
        # y = (x + RHS) / pho

        # Assuming 'pho' for this set of lines is related to 'start'
        pho_coeff = s

        # Calculate RHS for this strain state
        RHS = highest_taylor_factor / np.sqrt(6)

        # Avoid division by zero if pho_coeff is very close to zero
        if pho_coeff != 0:
            # Add expressions to the PyDemos graph using G.append()
            # Expression 1: y = (x - RHS) / pho_coeff
            G.append(f'y = (x - {RHS}) / {pho_coeff}')
            # Expression 2: y = (x + RHS) / pho_coeff
            G.append(f'y = (x + {RHS}) / {pho_coeff}')
        else:
            # If pho is zero, the equations become x = RHS and x = -RHS (vertical lines)
            G.append(f'x = {RHS}')
            G.append(f'x = {-RHS}')


    # Plot lines for varying 'phro'
    for phro_val, (sys, highest_taylor_factor) in M_by_phro.items():
        # Equations: -x/pho + y = RHS and x/pho - y = RHS
        # Rearranged to solve for y (assuming x on x-axis, y on y-axis):
        # y = RHS + x / pho
        # y = -RHS + x / pho

        # Assuming 'pho' for this set of lines is related to 'phro'
        pho_coeff = phro_val

        # Calculate RHS for this strain state
        RHS = highest_taylor_factor / np.sqrt(6)

        # Ensure pho_coeff is not zero to avoid division by zero in the equation
        if pho_coeff != 0:
             # Add expressions to the PyDemos graph using G.append()
            # Expression 1: y = RHS + x / pho_coeff
            G.append(f'y = {RHS} + x / {pho_coeff}')
            # Expression 2: y = -RHS + x / pho_coeff
            G.append(f'y = {-RHS} + x / {pho_coeff}')
        # Note: If pho_coeff is 0 here, the equations become y = RHS and y = -RHS (horizontal lines).
        # However, the strain tensor definition for phro includes 1/phro, so phro=0 is likely
        # a physically invalid or singular case you are already skipping.
        # If you need to handle the phro=0 case for plotting, you would add
        # G.append(f'y = {RHS}') and G.append(f'y = {-RHS}') here.

    # Plot for varing Z
    for z, (sys, highest_taylor_factor) in M_by_z.items():
        # Equations: x - phoy = RHS and -x + phoy = RHS
        # Rearranged to solve for y (assuming x on x-axis, y on y-axis):
        RHS= highest_taylor_factor / np.sqrt(6)
        if z !=0:
          G.append(f'y = (x - {RHS} * ({z} - 1)) / {z}')

          G.append(f'y = (x + {RHS} * ({z} - 1)) / {z}')



# The PyDemos graph will be generated when the 'with' block exits.
# You can then open the generated HTML file to view the graph in Desmos.

Avg value 2.535816083948781
Avg value 2.5330278604248315
Avg value 2.5345171791439935
Avg value 2.5381246733825824
Avg value 2.542772699744068
Avg value 2.548145331499472
Avg value 2.5543148781196883
Avg value 2.561256364008806
Avg value 2.568732975020588
Avg value 2.576617357878413
Avg value 2.584728645623069
Avg value 2.593163297457669
Avg value 2.6017140831324173
Avg value 2.610621554752687
Avg value 2.6198373613230803
Avg value 2.6293385212347116
Avg value 2.638979484333391
Avg value 2.6488588460142766
Avg value 2.658898870698191
Avg value 2.6691059484877213
Avg value 2.679422381317938
Avg value 2.689800126165269
Avg value 2.7002663538226725
Avg value 2.7108403144086988
Avg value 2.7214988181558977
Avg value 2.7322781360603714
Avg value 2.743140155040167
Avg value 2.7540496351382546
Avg value 2.765063204539506
Avg value 2.7761936151217617
Avg value 2.787364412163548
Avg value 2.798573614791289
Avg value 2.80978281741903
Avg value 2.8209920200467704
Avg value 2.832201222674511
Avg v

In [45]:

# --- PyDemos Plotting ---

with Graph('Yield Locus for Cube Polycrystal') as G:
    # Counters for each section
    start_nonzero_count = 0
    start_zero_count = 0
    phro_nonzero_count = 0
    z_nonzero_count = 0

    # Plot lines for varying 'start'
    for s, (sys, highest_taylor_factor) in M_by_start.items():
        # Equations: x - phoy = RHS and -x + phoy = RHS
        # Rearranged to solve for y (assuming x on x-axis, y on y-axis):
        # y = (x - RHS) / pho
        # y = (x + RHS) / pho

        # Assuming 'pho' for this set of lines is related to 'start'
        pho_coeff = s

        # Calculate RHS for this strain state
        RHS = highest_taylor_factor / np.sqrt(6)

        # Avoid division by zero if pho_coeff is very close to zero
        if not np.isclose(pho_coeff, 0):
            # Add expressions to the PyDemos graph using G.append()
            # Expression 1: y = (x - RHS) / pho_coeff
            G.append(f'y = (x - {RHS}) / {pho_coeff}')
            # Expression 2: y = (x + RHS) / pho_coeff
            G.append(f'y = (x + {RHS}) / {pho_coeff}')
            start_nonzero_count += 1 # Increment counter for non-zero pho_coeff
        else:
            # If pho is zero, the equations become x = RHS and x = -RHS (vertical lines)
            G.append(f'x = {RHS}')
            G.append(f'x = {-RHS}')
            start_zero_count += 1 # Increment counter for zero pho_coeff


    # Plot lines for varying 'phro'
    for phro_val, (sys, highest_taylor_factor) in M_by_phro.items():
        # Equations: -x/pho + y = RHS and x/pho - y = RHS
        # Rearranged to solve for y (assuming x on x-axis, y on y-axis):
        # y = RHS + x / pho
        # y = -RHS + x / pho

        # Assuming 'pho' for this set of lines is related to 'phro'
        pho_coeff = phro_val

        # Calculate RHS for this strain state
        RHS = highest_taylor_factor / np.sqrt(6)

        # Ensure pho_coeff is not zero to avoid division by zero in the equation
        if pho_coeff != 0:
             # Add expressions to the PyDemos graph using G.append()
            # Expression 1: y = RHS + x / pho_coeff
            G.append(f'y = {RHS} + x / {pho_coeff}')
            # Expression 2: y = -RHS + x / pho_coeff
            G.append(f'y = {-RHS} + x / {pho_coeff}')
            phro_nonzero_count += 1 # Increment counter for non-zero pho_coeff
        # Note: If pho_coeff is 0 here, the equations become y = RHS and y = -RHS (horizontal lines).
        # However, the strain tensor definition for phro includes 1/phro, so phro=0 is likely
        # a physically invalid or singular case you are already skipping.
        # If you need to handle the phro=0 case for plotting, you would add
        # G.append(f'y = {RHS}') and G.append(f'y = {-RHS}') here.


    # Plot for varing Z
    for z, (sys, highest_taylor_factor) in M_by_z.items():
        # Equations: x - phoy = RHS and -x + phoy = RHS
        # Rearranged to solve for y (assuming x on x-axis, y on y-axis):
        RHS= highest_taylor_factor / np.sqrt(6)
        if z !=0:
          G.append(f'y = (x - {RHS} * ({z} - 1)) / {z}')
          G.append(f'y = (x + {RHS} * ({z} - 1)) / {z}')
          z_nonzero_count += 1 # Increment counter for non-zero z


# Print the counts after the plotting is done
print(f"Number of times 'start' loop added non-zero pho_coeff equations: {start_nonzero_count}")
print(f"Number of times 'start' loop added zero pho_coeff equations: {start_zero_count}")
print(f"Number of times 'phro' loop added non-zero pho_coeff equations: {phro_nonzero_count}")
print(f"Number of times 'z' loop added non-zero z equations: {z_nonzero_count}")

# The PyDemos graph will be generated when the 'with' block exits.
# You can then open the generated HTML file to view the graph in Desmos.

Number of times 'start' loop added non-zero pho_coeff equations: 99
Number of times 'start' loop added zero pho_coeff equations: 1
Number of times 'phro' loop added non-zero pho_coeff equations: 100
Number of times 'z' loop added non-zero z equations: 100


In [26]:
# M_by_phro


{np.float64(1.0): ('BH_40', np.float64(3.1872784389423154))}